In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
from datetime import datetime

from config import *
from misc.utils import *
from data.generator import DataGenerator

In [2]:
import argparse
class Parser:

    def __init__(self):
        self.parser = argparse.ArgumentParser()
        self.set_arguments()

    def set_arguments(self):
        
        self.parser.add_argument('--work-type', type=str, help='to set work types e.g. gen-data, train ')
        self.parser.add_argument('--gpu', type=str, help='to set gpu ids to use')
        self.parser.add_argument('--gpu-mem-multiplier', type=int, help='to set gpu memory size (GB) ')
        
        self.parser.add_argument('--model', type=str, help='to set model to experiment')
        self.parser.add_argument('--task', type=str, help='to set tasks (e.g., non_iid_50, etc.)')
        self.parser.add_argument('--seed', type=int, help='to set seed')
        self.parser.add_argument('--num-rounds', type=int, help='to set number of rounds per task')
        self.parser.add_argument('--num-epochs', type=int, help='to set number of epochs per round')
        self.parser.add_argument('--batch-size', type=int, help='to set batch size')
        self.parser.add_argument('--train-mode',type=str, help='select mode (full,mask,kb)')

    def parse(self,arguments):
        args, unparsed  = self.parser.parse_known_args(args=arguments)
        if len(unparsed) != 0:
            raise SystemExit('Unknown argument: {}'.format(unparsed))
        return args

In [3]:
args_str = '''--gpu 0,1,2 \
					--work-type train \
					--model fedweit \
					--task non_iid_50 \
					--gpu-mem-multiplier 9 \
					--num-rounds 3 \
					--num-epochs 1 \
					--batch-size 100 \
					--seed 777 \
					--train-mode mask'''

In [4]:
args = set_data_config(Parser().parse(args_str.split()))

In [5]:
args = set_config(args)

In [6]:
os.environ['CUDA_VISIBLE_DEVICES']=args.gpu
if args.model == 'fedweit':
    from models.fedweit.server import Server
    server = Server(args)


2023-02-10 20:13:29.444327: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-02-10 20:13:36.026021: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:12:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.61GiB deviceMemoryBandwidth: 298.08GiB/s
2023-02-10 20:13:36.028910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 1 with properties: 
pciBusID: 0000:37:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.61GiB deviceMemoryBandwidth: 298.08GiB/s
2023-02-10 20:13:36.032346: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 2 with properties: 
pciBusID: 0000:86:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.61GiB deviceMemoryBandwidth: 298.08GiB/s
2023-02-10 20:13:36.033675: I tensorf

In [ ]:
# server.run()
# federated server 모듈의 run()함수 수행

In [7]:
for lid, shape in enumerate(server.nets.shapes):
    shape = np.concatenate([server.nets.shapes[lid],[int(round(server.args.num_clients*server.args.frac_clients))]], axis=0)
    print('layer_id : ',lid)
    print('shapes : ',server.nets.shapes[lid])
    print('num_client : ',[int(round(server.args.num_clients*server.args.frac_clients))])
    print(shape)
    print('#############')
    

layer_id :  0
shapes :  (5, 5, 3, 20)
num_client :  [3]
[ 5  5  3 20  3]
#############
layer_id :  1
shapes :  (5, 5, 20, 50)
num_client :  [3]
[ 5  5 20 50  3]
#############
layer_id :  2
shapes :  (3200, 800)
num_client :  [3]
[3200  800    3]
#############
layer_id :  3
shapes :  (800, 500)
num_client :  [3]
[800 500   3]
#############


In [14]:
type(server.nets.decomposed_variables['from_kb'])

dict

In [19]:
state = np_load(os.path.join('/home/proj01/FCL/output/states/20230127-1258-fedweit-non_iid_50/', '{}_net.npy'.format(0))).item()

In [26]:
state.keys()

dict_keys(['client_id', 'decomposed_weights', 'heads_weights'])

In [36]:
for k,v in state.items():
    print(k)
    if type(v) == dict:
        print('------------')
        for ki,vi in v.items():
            print(ki)
        print('------------')
    else:
        continue
        print(v)

client_id
decomposed_weights
------------
shared
adaptive
mask
bias
from_kb
atten
------------
heads_weights


In [82]:
# shared : 4,(5,5,3,20)
# adaptive : task수만큼 10, 4,(5,5,3,20)
# mask : task수만큼 10, 4,20
# bias : task수만큼 10, 4,20
# from_kb : task수만큼 10, 4,(5,5,3,20,3) 
# atten : task수만큼 10, 4, 3
#여기서 3은 아마 클라이언트 수를 뜻함 4는 모델의 layer 수
state['decomposed_weights']['atten'][0][0].shape

(3,)

In [13]:
for lid, shape in enumerate(parse.nets.shapes):
    shape = np.concatenate([self.nets.shapes[lid],[int(round(parse.args.num_clients*parse.args.))]])

'mask'